In [ ]:

import irt_mt_dev.utils as utils
import numpy as np
import os
import subset2evaluate.evaluate
import subset2evaluate.select_subset
import tqdm

os.chdir("/home/vilda/irt-mt-dev")

def benchmark_method(repetitions=10, kwargs_dict={}):
	data_old_all = list(utils.load_data_wmt_all(normalize=True).values())[:9]
	points_y_acc = []
	points_y_clu = []

	# run multiple times to smooth variance
	for data_old in data_old_all:
		for _ in range(repetitions):
			while True:
				try:
					(_, clu_new), acc_new = subset2evaluate.evaluate.run_evaluate_topk(
						data_old,
						subset2evaluate.select_subset.run_select_subset(data_old, **kwargs_dict),
						metric="human"
					)
					break
				except Exception as e:
					print(e)
					pass
			points_y_acc.append(acc_new)
			points_y_clu.append(clu_new)

		# print(f"- ACC: {np.average(acc_new):.2%} | CLU: {np.average(clu_new):.2f}")
	print(f"ACC: {np.average(points_y_acc):.2%} | CLU: {np.average(points_y_clu):.2f}")

In [ ]:
print("Random")
benchmark_method(repetitions=10, kwargs_dict={"method": "random"})

In [ ]:
print("MetricX-23 avg")
benchmark_method(repetitions=1, kwargs_dict={"method": "avg", "metric": "MetricX-23"})
print("MetricX-23 var")
benchmark_method(repetitions=1, kwargs_dict={"method": "var", "metric": "MetricX-23"})

In [ ]:
# done with early stopping
print("IRT Fisher Information Content")
benchmark_method(repetitions=5, kwargs_dict={"method": "irt_fic", "metric": "MetricX-23", "model": "scalar"})

In [ ]:
print("PyIRT Fisher Information Content")

# config = py_irt.config.IrtConfig(
#     model_type='4pl',
#     log_every=100,
#     # dropout=0,
#     seed=0,
#     # TODO: see what this does
#     deterministic=True,
# )

benchmark_method(repetitions=3, kwargs_dict={"method": "pyirt_fic", "metric": "MetricX-23", 'deterministic': True, 'epochs': 1000, 'model_type': '4pl', 'dropout': 0.5, 'priors': 'vague'})